In [8]:
import torch
import os
import torch as t
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import cv2

import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [9]:
from skimage.transform import resize


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google'

In [5]:
class ASL(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, transform=None):

        self.dataset_file = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.dataset_file)

    def __getitem__(self, idx):
        img_name = self.dataset_file.iloc[idx,1]
        img_name = img_name.split('\t')
        img_name=img_name[0]
#         print(img_name)
        for frames in range(1,31,1):
            if frames is 1:
#                 print(img_name)
                y = cv2.imread(img_name+"/"+str(frames)+".jpg")
                y = resize(y, (100, 100))
#                 print(y.shape)
#                 print(img_name+"/"+str(frames)+".jpg")
#                 plt.imshow(y)
#                 plt.show()
                y=y.reshape(1,100,100,3)
            else:
                z=cv2.imread(img_name+"/"+str(frames)+".jpg")
                z=resize(z, (100, 100))
                z=z.reshape(1,100,100,3)
                y=np.concatenate((y, z), axis=0)
        
        video = t.tensor(y,dtype=torch.float)
        video.transpose_(3,0)
        video.transpose_(1,3)
        video.transpose_(2,3)
#        
        label = self.dataset_file.iloc[idx, 3]
#         print(label)
#         label = label.split('\t')
#         label = label[2]0
        sample = {'video': video, 'label': label}

        if self.transform:
            sample = self.transform(sample)
        return sample

In [6]:

Sign_dataset = ASL(csv_file='/content/gdrive/My Drive/Dataset/most.csv')


for i in range(len(Sign_dataset)):
    sample = Sign_dataset[i]
#     sample['video']=sample['video'].reshape(3,30,440,440)
    sample['video'].transpose_(3,0)
    sample['video'].transpose_(0,2)
    sample['video'].transpose_(0,1)
    print(sample['video'].shape, sample['label'])
    plt.imshow(sample['video'][0])
    plt.show() 
    if i == 3:
      break

FileNotFoundError: File b'/content/gdrive/My Drive/Dataset/most.csv' does not exist

In [0]:
dataloader = DataLoader(Sign_dataset, batch_size=4,
                        shuffle=True, num_workers=4)

for i_batch, sample_batched in enumerate(dataloader):
    
    print(i_batch, sample_batched['video'].type(),sample_batched['label'])
    if i_batch == 3:
        break

0 torch.FloatTensor tensor([4, 4, 5, 9])
1 torch.FloatTensor tensor([2, 0, 2, 5])
2 torch.FloatTensor tensor([9, 2, 8, 0])
3 torch.FloatTensor tensor([2, 2, 2, 1])


In [10]:
import torch.nn as nn
import torch.optim as optim


class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.block1 = nn.Sequential(
            nn.Conv3d(3, 32, kernel_size=(3, 5, 5), stride=(1, 2, 2), dilation=(1, 1, 1), padding=(1, 2, 2)),
            nn.BatchNorm3d(32),
            nn.ReLU(inplace=True),
            nn.Dropout3d(p=0.2),
        )

        self.block2 = nn.Sequential(
            nn.Conv3d(32, 64, kernel_size=(3, 3, 3), stride=1, dilation=(1, 1, 1), padding=(1, 1, 1)),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True),
            nn.Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 2, 2), dilation=(1, 1, 1), padding=(1, 1, 1)),
            nn.BatchNorm3d(128),
            nn.ReLU(inplace=True),
            nn.Dropout3d(p=0.2),
        )

        self.block3 = nn.Sequential(
            nn.Conv3d(128, 128, kernel_size=(3, 3, 3), stride=1, dilation=(1, 1, 1), padding=(1, 1, 1)),
            nn.BatchNorm3d(128),
            nn.ReLU(inplace=True),
            nn.Conv3d(128, 128, kernel_size=(3, 3, 3), stride=1, dilation=(1, 1, 1), padding=(1, 1, 1)),
            nn.BatchNorm3d(128),
            nn.ReLU(inplace=True),
            nn.Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 2, 2), dilation=(1, 1, 1), padding=(1, 1, 1)),
            nn.BatchNorm3d(256),
            nn.ReLU(inplace=True),
            nn.Dropout3d(p=0.2),
        )

        self.block4 = nn.Sequential(
            nn.Conv3d(256, 256, kernel_size=(3, 3, 3), stride=1, dilation=(1, 1, 1), padding=(1, 1, 1)),
            nn.BatchNorm3d(256),
            nn.ReLU(inplace=True),
            nn.Conv3d(256, 256, kernel_size=(3, 3, 3), stride=1, dilation=(1, 1, 1), padding=(1, 1, 1)),
            nn.BatchNorm3d(256),
            nn.ReLU(inplace=True),
            nn.Conv3d(256, 512, kernel_size=(3, 3, 3), stride=(1, 2, 2), dilation=(1, 1, 1), padding=(1, 1, 1)),
            nn.BatchNorm3d(512),
            nn.ReLU(inplace=True),
            nn.Dropout3d(p=0.2),
        )

        self.block5 = nn.Sequential(
            nn.Conv3d(512, 512, kernel_size=(3, 3, 3), stride=1, dilation=(1, 1, 1), padding=(1, 1, 1)),
            nn.BatchNorm3d(512),
            nn.ReLU(inplace=True),
            nn.Conv3d(512, 512, kernel_size=(3, 3, 3), stride=(1, 2, 2), dilation=(1, 1, 1), padding=(1, 1, 1)),
            nn.BatchNorm3d(512),
            nn.ReLU(inplace=True),

        )
        self.block6 = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 120),
            nn.ReLU(inplace=True),
            nn.Linear(120, 10)
        )
#         self.block6 = nn.Sequential(
#             nn.Linear(1024*4*4*30, 512),
#             nn.Linear(512, 84),
#             nn.Linear(84, 10)
#         )

    def forward(self, x):
        # get convolution column features
#         print(x.size())
        x = self.block1(x)
#         for y in range(32):
#         z=x[0,1,0]
#         z=(z*255).int()
#         plt.imshow(z)
#         plt.show()
#         print(x.size())
        x = self.block2(x)
#         print(x.size())
#         for y in range(128):
#             z=x[0,y,0]
#             z=(z*255).int()
#             plt.imshow(z)
#             plt.show()
        x = self.block3(x)
#         print(x.size())
#         for y in range(256):
#             z=x[0,y,0]
#             z=(z*255).int()
#             plt.imshow(z)
#             plt.show()
#         x = self.block4(x)
# #         print(x.size())
# #         for y in range(512):
# #             z=x[0,y,0]
# #             z=(z*255).int()
# #             plt.imshow(z)
# #             plt.show()
#         x = self.block5(x)
# #         print(x.size())
#         for y in range(1024):
#             z=x[0,y,0]
#             print(z)
#             z=(z*255).int()
#             plt.imshow(z)
#             plt.show()
#         x = self.block6(x)
#         print(x.size())
#         x = x.view(-1, 12 * 4 * 4)
        # averaging features in time dimension
        x = x.mean(-1).mean(-1).mean(-1)
#         x = x.view(-1, 512 * 30 * 4*4)
#         x = self.block6(x)

#         print(x.size())
        return x


input_tensor = torch.autograd.Variable(torch.rand(5, 3, 72, 84, 84))
model = Model().cuda()
output = model(input_tensor.cuda())


# model = Model()

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


# for epoch in range(20):  

#     running_loss = 0.0
#     for i, data in enumerate(dataloader, 0):
        
#         inputs, labels = data['video'],data['label']
#         inputs = torch.autograd.Variable(inputs)
#         labels=t.tensor(np.array(list(map(int, labels))),dtype=torch.long)
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         print(labels.shape)
#         print(outputs.shape)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
#         print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss))
#         running_loss = 0.0

# print('Finished Training')

RuntimeError: CUDA error: out of memory